In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import linregress
import gc
import warnings
warnings.filterwarnings('ignore')

In [ ]:
train = pd.read_pickle('Data/test_data.pkl')
train['time'] = pd.to_datetime(train['S_2'])                                                                                                                                                                                                    
train['time']  = train['time'].astype(np.int64) / 10**18
enc_train = pd.read_pickle('Data/enc_test.pkl')
enc_train['time'] = train['time']

del train; gc.collect()
enc_train.shape

In [ ]:
def slope(df):
    slope_df = pd.DataFrame(index=df.index.unique())
    num_columns = df.columns.to_list()
    num_columns.remove('time')
    for i in num_columns:
        print(i, end=', ')
        slope_df['enc_slope_' + i] = df.groupby('customer_ID').apply(lambda v: linregress(v.time, v[i])[0])

    return slope_df

In [ ]:
slope_df = slope(enc_train)

In [ ]:
enc_train.drop(['time'], axis=1, inplace=True)

In [ ]:
def aggregator(df):
    operation_list = []
    for i in df.columns:
        operation_list.append(['mean', 'std', 'min', 'max', 'last'])
    dic_numerical = dict(zip(df.columns, operation_list))
    df_agg_num = df.groupby('customer_ID').agg(dic_numerical)
    df_agg_num.columns = ['_'.join(x) for x in df_agg_num.columns]

    return df_agg_num

In [ ]:
agg_df = aggregator(enc_train)

In [ ]:
def feature_engineering(df):
    last_minus_first = df.groupby('customer_ID').apply(lambda x: x.iloc[-1] - x.iloc[0])
    last_minus_first.columns = ['last-first_' + c for c in last_minus_first]
    print('last_minus_first Done')

    last_dividedby_first = df.groupby('customer_ID').apply(lambda x: x.iloc[-1] / (x.iloc[0] + 0.0000001))
    last_dividedby_first.columns = ['last/first_' + c for c in last_dividedby_first]
    print('last_dividedby_first Done')

    last_dividedby_ave = df.groupby('customer_ID').apply(lambda x: x.iloc[-1] / (x.mean() + 0.0000001))
    last_dividedby_ave.columns = ['last/ave_' + c for c in last_dividedby_ave]
    print('last_dividedby_ave Done')

    last_minus_ave = df.groupby('customer_ID').apply(lambda x: x.iloc[-1] - x.mean())
    last_minus_ave.columns = ['last-ave_' + c for c in last_minus_ave]
    print('last_minus_ave Done')

    first = df.groupby('customer_ID').first()
    first.columns = ['first_' + c for c in first]
    print('first Done')

    last_minus2_diff = df.groupby('customer_ID').apply(lambda x: x.iloc[-1] - x.iloc[-2] if len(x) > 1 else x.iloc[-1] - x.iloc[0])
    last_minus2_diff.columns = ['diff_last_-2_' + c for c in last_minus2_diff]
    print('diff_last_-2 Done')

    return pd.concat([last_minus_first, last_dividedby_first, last_dividedby_ave, last_minus_ave, first, last_minus2_diff], axis=1)

In [ ]:
lag_df = feature_engineering(enc_train)

In [ ]:
all = pd.concat([slope_df, agg_df, lag_df], axis=1)
all.to_pickle('Data/autoenc_test_features.pkl')

In [ ]:
all = pd.read_pickle('Data/autoenc_test_features.pkl')

In [ ]:
useless_0 = [
    "last-first_R_22",
    "NA_D_120",
    "last-first_R_23",
    "NA_B_42",
    "date_month_max",
    "NA_D_122",
    "NA_D_123",
    "NA_D_124",
    "NA_D_125",
    "NA_D_126",
    "NA_D_127",
    "NA_D_129",
    "NA_D_117",
    "NA_B_41",
    "NA_D_116",
    "date_month_last",
    "NA_D_114",
    "NA_D_113",
    "last-first_D_109",
    "NA_R_28",
    "NA_B_40",
    "NA_D_136",
    "NA_D_137",
    "NA_D_138",
    "NA_D_139",
    "NA_D_140",
    "NA_D_143",
    "NA_D_112",
    "NA_D_145",
    "B_41_min",
    "last-first_D_94",
    "last-first_D_93",
    "NA_D_135",
    "slope_D_93",
    "D_65_min",
    "diff_last_-2_R_20",
    "diff_last_-2_D_82",
    "D_143_max",
    "diff_last_-2_D_83",
    "diff_last_-2_D_86",
    "diff_last_-2_D_87",
    "slope_D_109",
    "diff_last_-2_R_17",
    "diff_last_-2_R_18",
    "diff_last_-2_D_88",
    "slope_D_110",
    "diff_last_-2_B_31",
    "diff_last_-2_R_19",
    "slope_B_42",
    "diff_last_-2_S_20",
    "slope_R_28",
    "last-first_D_88",
    "diff_last_-2_R_21",
    "diff_last_-2_D_111",
    "diff_last_-2_D_109",
    "diff_last_-2_R_23",
    "diff_last_-2_D_92",
    "diff_last_-2_D_93",
    "diff_last_-2_D_94",
    "diff_last_-2_R_24",
    "diff_last_-2_R_25",
    "diff_last_-2_D_96",
    "D_116_last",
    "D_116_nunique",
    "diff_last_-2_R_26",
    "R_2_min",
    "diff_last_-2_D_81",
    "diff_last_-2_D_127",
    "slope_D_94",
    "diff_last_-2_R_9",
    "last-first_R_18",
    "last-first_R_17",
    "last-first_D_87",
    "last-first_D_86",
    "D_88_last_round2",
    "R_5_min",
    "B_42_std",
    "R_4_min",
    "slope_D_87",
    "diff_last_-2_D_143",
    "R_28_mean",
    "slope_R_18",
    "R_28_std",
    "R_28_min",
    "R_28_max",
    "diff_last_-2_D_139",
    "R_28_last",
    "S_6_min",
    "diff_last_-2_D_72",
    "D_135_min",
    "D_136_min",
    "slope_D_88",
    "diff_last_-2_R_28",
    "D_137_min",
    "D_137_last",
    "D_138_min",
    "slope_R_23",
    "diff_last_-2_B_41",
    "diff_last_-2_D_129",
    "NA_D_111",
    "NA_R_3",
    "NA_D_109",
    "D_94_min",
    "last-ave_D_93",
    "last-ave_R_23",
    "last-ave_D_88",
    "last-ave_R_18",
    "last-ave_D_87",
    "D_96_min",
    "R_25_last",
    "R_25_min",
    "R_24_last",
    "R_24_min",
    "D_94_last",
    "D_94_max",
    "D_94_std",
    "NA_B_1",
    "D_93_last",
    "D_93_max",
    "D_93_min",
    "D_93_std",
    "D_93_mean",
    "D_92_last",
    "D_92_min",
    "R_23_last",
    "R_23_max",
    "R_23_min",
    "R_23_std",
    "R_23_mean",
    "last-ave_D_94",
    "last/first_R_28",
    "last/first_B_39",
    "last/first_D_110",
    "NA_S_2",
    "last-first_D_139",
    "D_125_min",
    "cat_first_D_116",
    "D_123_min",
    "D_111_min",
    "D_109_last",
    "cat_-2_D_116",
    "D_109_max",
    "D_109_min",
    "D_109_std",
    "D_109_mean",
    "D_108_last",
    "D_108_min",
    "last/first_S_18",
    "last-ave_R_28",
    "last/first_D_87",
    "last/first_R_18",
    "last/first_D_88",
    "last/first_R_23",
    "last/first_D_93",
    "last/first_D_94",
    "last/first_R_25",
    "last-ave_D_109",
    "last/first_D_109",
    "R_22_last",
    "R_22_min",
    "R_7_min",
    "first_D_108",
    "D_87_last",
    "D_87_max",
    "R_15_min",
    "R_15_last",
    "D_87_min",
    "D_87_std",
    "first_D_96",
    "first_R_25",
    "first_D_94",
    "first_D_93",
    "first_R_23",
    "first_R_22",
    "first_D_89",
    "first_R_21",
    "D_87_mean",
    "first_B_31",
    "first_R_18",
    "first_R_17",
    "first_D_87",
    "first_D_86",
    "first_R_13",
    "D_86_min",
    "R_16_min",
    "S_18_last",
    "S_18_min",
    "R_17_min",
    "first_D_109",
    "last/ave_D_87",
    "first_D_111",
    "last/ave_R_18",
    "last/ave_D_88",
    "last/ave_R_23",
    "last/ave_D_93",
    "R_8_min",
    "last/ave_D_94",
    "last/ave_R_25",
    "R_20_min",
    "S_20_min",
    "last/ave_D_109",
    "R_10_min",
    "R_19_last",
    "last/ave_R_28",
    "R_13_min",
    "B_31_max",
    "first_D_138",
    "first_R_28",
    "D_88_last",
    "D_88_mean",
    "R_18_last",
    "R_18_max",
    "R_18_min",
    "R_18_std",
    "R_14_min",
    "R_18_mean",
    "NA_D_39",
    "NA_B_2",
    "NA_D_108",
    "NA_B_28",
    "NA_R_18",
    "NA_R_17",
    "NA_D_87",
    "NA_D_86",
    "NA_S_18",
    "NA_B_30",
    "NA_R_16",
    "NA_D_84",
    "NA_R_15",
    "NA_R_14",
    "NA_D_83",
    "NA_R_13",
    "NA_R_12",
    "NA_R_1",
    "NA_S_17",
    "NA_D_82",
    "NA_D_81",
    "NA_B_27",
    "NA_R_11",
    "NA_R_10",
    "NA_D_80",
    "NA_S_16",
    "NA_R_9",
    "NA_R_8",
    "NA_D_79",
    "NA_D_78",
    "NA_D_88",
    "NA_B_31",
    "NA_S_19",
    "NA_R_19",
    "NA_B_38",
    "NA_R_26",
    "NA_B_37",
    "NA_B_36",
    "NA_D_107",
    "NA_D_106",
    "last-first_D_110",
    "NA_D_103",
    "NA_S_26",
    "NA_S_23",
    "NA_D_96",
    "NA_R_25",
    "NA_R_24",
    "NA_D_94",
    "NA_D_93",
    "NA_D_92",
    "NA_D_91",
    "NA_R_23",
    "NA_R_22",
    "NA_D_89",
    "NA_B_33",
    "NA_R_21",
    "NA_R_20",
    "NA_S_20",
    "NA_B_32",
    "NA_B_26",
    "NA_B_25",
    "last-first_B_39",
    "NA_D_59",
    "NA_D_58",
    "NA_R_5",
    "last-first_D_127",
    "NA_S_8",
    "NA_B_12",
    "NA_R_4",
    "NA_D_54",
    "NA_S_6",
    "NA_B_11",
    "NA_S_5",
    "NA_B_10",
    "NA_B_9",
    "NA_D_51",
    "NA_B_7",
    "NA_B_6",
    "NA_D_49",
    "last-first_R_28",
    "NA_D_47",
    "NA_R_2",
    "NA_B_5",
    "NA_D_45",
    "NA_B_4",
    "NA_D_44",
    "NA_B_3",
    "NA_D_41",
    "NA_B_14",
    "NA_D_60",
    "NA_R_7",
    "D_127_min",
    "NA_B_24",
    "NA_D_75",
    "NA_D_74",
    "NA_P_4",
    "NA_B_23",
    "NA_S_15",
    "NA_D_72",
    "NA_D_71",
    "NA_D_70",
    "NA_B_22",
    "NA_B_21",
    "NA_S_13",
    "NA_R_6",
    "NA_S_12",
    "NA_D_68",
    "NA_B_20",
    "NA_D_66",
    "NA_B_19",
    "NA_B_18",
    "NA_B_16",
    "NA_D_65",
    "NA_D_64",
    "NA_D_63",
    "NA_S_11",
    "NA_B_15",
    "D_88_min",
]
useless_1 = [
    "last-first_R_22",
    "NA_D_120",
    "last-first_R_23",
    "NA_B_42",
    "date_month_max",
    "NA_D_122",
    "NA_D_123",
    "NA_D_124",
    "NA_D_125",
    "NA_D_126",
    "NA_D_127",
    "NA_D_129",
    "NA_D_117",
    "NA_B_41",
    "NA_D_116",
    "date_month_last",
    "NA_D_114",
    "NA_D_113",
    "last-first_D_109",
    "NA_R_28",
    "NA_B_40",
    "NA_D_136",
    "NA_D_137",
    "NA_D_138",
    "NA_D_139",
    "NA_D_140",
    "NA_D_143",
    "NA_D_112",
    "NA_D_145",
    "B_41_min",
    "last-first_D_94",
    "last-first_D_93",
    "NA_D_135",
    "slope_D_93",
    "D_65_min",
    "diff_last_-2_R_20",
    "diff_last_-2_D_82",
    "D_143_max",
    "diff_last_-2_D_83",
    "diff_last_-2_D_86",
    "diff_last_-2_D_87",
    "slope_D_109",
    "diff_last_-2_R_17",
    "diff_last_-2_R_18",
    "diff_last_-2_D_88",
    "slope_D_110",
    "diff_last_-2_B_31",
    "diff_last_-2_R_19",
    "slope_B_42",
    "diff_last_-2_S_20",
    "slope_R_28",
    "last-first_D_88",
    "diff_last_-2_R_21",
    "diff_last_-2_D_111",
    "diff_last_-2_D_109",
    "diff_last_-2_R_23",
    "diff_last_-2_D_92",
    "diff_last_-2_D_93",
    "diff_last_-2_D_94",
    "diff_last_-2_R_24",
    "diff_last_-2_R_25",
    "diff_last_-2_D_96",
    "D_116_last",
    "D_116_nunique",
    "diff_last_-2_R_26",
    "R_2_min",
    "diff_last_-2_D_81",
    "diff_last_-2_D_127",
    "slope_D_94",
    "diff_last_-2_R_9",
    "last-first_R_18",
    "last-first_R_17",
    "last-first_D_87",
    "last-first_D_86",
    "D_88_last_round2",
    "R_5_min",
    "B_42_std",
    "R_4_min",
    "slope_D_87",
    "diff_last_-2_D_143",
    "R_28_mean",
    "slope_R_18",
    "R_28_std",
    "R_28_min",
    "R_28_max",
    "diff_last_-2_D_139",
    "R_28_last",
    "S_6_min",
    "diff_last_-2_D_72",
    "D_135_min",
    "D_136_min",
    "slope_D_88",
    "diff_last_-2_R_28",
    "D_137_min",
    "D_137_last",
    "D_138_min",
    "slope_R_23",
    "diff_last_-2_B_41",
    "diff_last_-2_D_129",
    "NA_D_111",
    "NA_R_3",
    "NA_D_109",
    "D_94_min",
    "last-ave_D_93",
    "last-ave_R_23",
    "last-ave_D_88",
    "last-ave_R_18",
    "last-ave_D_87",
    "D_96_min",
    "R_25_last",
    "R_25_min",
    "R_24_last",
    "R_24_min",
    "D_94_last",
    "D_94_max",
    "D_94_std",
    "NA_B_1",
    "D_93_last",
    "D_93_max",
    "D_93_min",
    "D_93_std",
    "D_93_mean",
    "D_92_last",
    "D_92_min",
    "R_23_last",
    "R_23_max",
    "R_23_min",
    "R_23_std",
    "R_23_mean",
    "last-ave_D_94",
    "last/first_R_28",
    "last/first_B_39",
    "last/first_D_110",
    "NA_S_2",
    "last-first_D_139",
    "D_125_min",
    "cat_first_D_116",
    "D_123_min",
    "D_111_min",
    "D_109_last",
    "cat_-2_D_116",
    "D_109_max",
    "D_109_min",
    "D_109_std",
    "D_109_mean",
    "D_108_last",
    "D_108_min",
    "last/first_S_18",
    "last-ave_R_28",
    "last/first_D_87",
    "last/first_R_18",
    "last/first_D_88",
    "last/first_R_23",
    "last/first_D_93",
    "last/first_D_94",
    "last/first_R_25",
    "last-ave_D_109",
    "last/first_D_109",
    "R_22_last",
    "R_22_min",
    "R_7_min",
    "first_D_108",
    "D_87_last",
    "D_87_max",
    "R_15_min",
    "R_15_last",
    "D_87_min",
    "D_87_std",
    "first_D_96",
    "first_R_25",
    "first_D_94",
    "first_D_93",
    "first_R_23",
    "first_R_22",
    "first_D_89",
    "first_R_21",
    "D_87_mean",
    "first_B_31",
    "first_R_18",
    "first_R_17",
    "first_D_87",
    "first_D_86",
    "first_R_13",
    "D_86_min",
    "R_16_min",
    "S_18_last",
    "S_18_min",
    "R_17_min",
    "first_D_109",
    "last/ave_D_87",
    "first_D_111",
    "last/ave_R_18",
    "last/ave_D_88",
    "last/ave_R_23",
    "last/ave_D_93",
    "R_8_min",
    "last/ave_D_94",
    "last/ave_R_25",
    "R_20_min",
    "S_20_min",
    "last/ave_D_109",
    "R_10_min",
    "R_19_last",
    "last/ave_R_28",
    "R_13_min",
    "B_31_max",
    "first_D_138",
    "first_R_28",
    "D_88_last",
    "D_88_mean",
    "R_18_last",
    "R_18_max",
    "R_18_min",
    "R_18_std",
    "R_14_min",
    "R_18_mean",
    "NA_D_39",
    "NA_B_2",
    "NA_D_108",
    "NA_B_28",
    "NA_R_18",
    "NA_R_17",
    "NA_D_87",
    "NA_D_86",
    "NA_S_18",
    "NA_B_30",
    "NA_R_16",
    "NA_D_84",
    "NA_R_15",
    "NA_R_14",
    "NA_D_83",
    "NA_R_13",
    "NA_R_12",
    "NA_R_1",
    "NA_S_17",
    "NA_D_82",
    "NA_D_81",
    "NA_B_27",
    "NA_R_11",
    "NA_R_10",
    "NA_D_80",
    "NA_S_16",
    "NA_R_9",
    "NA_R_8",
    "NA_D_79",
    "NA_D_78",
    "NA_D_88",
    "NA_B_31",
    "NA_S_19",
    "NA_R_19",
    "NA_B_38",
    "NA_R_26",
    "NA_B_37",
    "NA_B_36",
    "NA_D_107",
    "NA_D_106",
    "last-first_D_110",
    "NA_D_103",
    "NA_S_26",
    "NA_S_23",
    "NA_D_96",
    "NA_R_25",
    "NA_R_24",
    "NA_D_94",
    "NA_D_93",
    "NA_D_92",
    "NA_D_91",
    "NA_R_23",
    "NA_R_22",
    "NA_D_89",
    "NA_B_33",
    "NA_R_21",
    "NA_R_20",
    "NA_S_20",
    "NA_B_32",
    "NA_B_26",
    "NA_B_25",
    "last-first_B_39",
    "NA_D_59",
    "NA_D_58",
    "NA_R_5",
    "last-first_D_127",
    "NA_S_8",
    "NA_B_12",
    "NA_R_4",
    "NA_D_54",
    "NA_S_6",
    "NA_B_11",
    "NA_S_5",
    "NA_B_10",
    "NA_B_9",
    "NA_D_51",
    "NA_B_7",
    "NA_B_6",
    "NA_D_49",
    "last-first_R_28",
    "NA_D_47",
    "NA_R_2",
    "NA_B_5",
    "NA_D_45",
    "NA_B_4",
    "NA_D_44",
    "NA_B_3",
    "NA_D_41",
    "NA_B_14",
    "NA_D_60",
    "NA_R_7",
    "D_127_min",
    "NA_B_24",
    "NA_D_75",
    "NA_D_74",
    "NA_P_4",
    "NA_B_23",
    "NA_S_15",
    "NA_D_72",
    "NA_D_71",
    "NA_D_70",
    "NA_B_22",
    "NA_B_21",
    "NA_S_13",
    "NA_R_6",
    "NA_S_12",
    "NA_D_68",
    "NA_B_20",
    "NA_D_66",
    "NA_B_19",
    "NA_B_18",
    "NA_B_16",
    "NA_D_65",
    "NA_D_64",
    "NA_D_63",
    "NA_S_11",
    "NA_B_15",
    "D_88_min",
]
useless_2 = [
    "D_136_min",
    "S_18_last",
    "R_23_min",
    "R_23_max",
    "R_23_last",
    "D_137_min",
    "D_92_min",
    "D_92_last",
    "D_87_std",
    "D_93_std",
    "D_87_mean",
    "B_31_max",
    "last/ave_D_92",
    "D_93_min",
    "D_87_min",
    "last/ave_D_93",
    "D_93_max",
    "D_93_last",
    "D_94_mean",
    "S_18_min",
    "D_65_min",
    "D_94_min",
    "D_94_max",
    "D_94_last",
    "first_R_4",
    "last/ave_D_94",
    "R_24_min",
    "last/ave_D_109",
    "R_23_std",
    "R_23_mean",
    "D_87_max",
    "R_20_min",
    "R_4_min",
    "last/ave_R_18",
    "D_88_min",
    "last/ave_R_28",
    "last/ave_D_87",
    "last/ave_S_18",
    "R_5_min",
    "D_88_mean",
    "R_18_last",
    "S_20_min",
    "R_18_max",
    "R_18_min",
    "R_18_std",
    "last/ave_D_88",
    "R_18_mean",
    "D_138_min",
    "R_21_min",
    "last/ave_R_22",
    "D_89_min",
    "S_6_min",
    "D_139_last",
    "R_17_min",
    "R_22_min",
    "D_87_last",
    "R_22_last",
    "last/ave_R_23",
    "R_24_last",
    "last/first_R_25",
    "last/first_R_28",
    "D_111_min",
    "D_123_min",
    "last-first_D_109",
    "last-first_D_110",
    "diff_last_-2_B_41",
    "diff_last_-2_D_127",
    "last-first_B_42",
    "last-first_R_28",
    "B_41_min",
    "B_42_std",
    "B_42_max",
    "D_109_last",
    "D_109_max",
    "D_109_min",
    "D_109_std",
    "D_109_mean",
    "diff_last_-2_D_111",
    "D_108_min",
    "diff_last_-2_D_109",
    "last-first_D_96",
    "diff_last_-2_R_28",
    "last-first_R_25",
    "last-first_R_24",
    "date_month_last",
    "D_127_min",
    "date_month_min",
    "D_125_min",
    "last-first_R_15",
    "last-first_D_87",
    "last-first_R_18",
    "last-first_D_88",
    "last-first_R_19",
    "last-first_S_20",
    "diff_last_-2_D_145",
    "diff_last_-2_D_143",
    "last-first_R_22",
    "last-first_R_23",
    "diff_last_-2_D_139",
    "last-first_D_93",
    "last-first_D_94",
    "diff_last_-2_D_108",
    "diff_last_-2_R_26",
    "R_25_min",
    "diff_last_-2_D_86",
    "last/first_R_23",
    "diff_last_-2_R_19",
    "last/first_D_93",
    "diff_last_-2_B_31",
    "diff_last_-2_D_88",
    "diff_last_-2_R_18",
    "diff_last_-2_R_17",
    "diff_last_-2_D_87",
    "last/first_D_94",
    "R_2_min",
    "D_135_min",
    "diff_last_-2_R_13",
    "last/first_D_109",
    "last/first_D_110",
    "diff_last_-2_D_82",
    "diff_last_-2_D_81",
    "D_96_min",
    "R_25_last",
    "diff_last_-2_R_21",
    "last/first_R_22",
    "diff_last_-2_D_89",
    "diff_last_-2_R_22",
    "R_28_mean",
    "R_28_std",
    "R_28_min",
    "R_28_max",
    "R_28_last",
    "last/first_R_15",
    "last/first_S_18",
    "last/first_D_87",
    "last/first_R_17",
    "last/first_R_18",
    "diff_last_-2_D_96",
    "diff_last_-2_R_25",
    "last/first_D_88",
    "diff_last_-2_D_94",
    "diff_last_-2_D_93",
    "diff_last_-2_D_92",
    "diff_last_-2_R_23",
    "R_16_min",
    "last-ave_R_28",
    "NA_D_145",
    "R_15_last",
    "NA_B_18",
    "R_13_min",
    "NA_B_16",
    "NA_D_65",
    "NA_D_64",
    "NA_D_63",
    "first_D_108",
    "NA_S_11",
    "NA_B_15",
    "first_D_109",
    "NA_D_60",
    "NA_D_59",
    "NA_B_14",
    "first_D_111",
    "NA_D_58",
    "NA_R_5",
    "slope_R_28",
    "NA_S_8",
    "NA_B_12",
    "first_R_28",
    "NA_R_4",
    "NA_B_19",
    "NA_D_66",
    "NA_B_20",
    "NA_P_4",
    "NA_D_78",
    "NA_B_26",
    "NA_B_25",
    "first_R_25",
    "NA_R_7",
    "NA_B_24",
    "first_D_96",
    "NA_D_75",
    "NA_D_74",
    "NA_B_23",
    "NA_D_68",
    "NA_S_15",
    "NA_D_72",
    "NA_D_71",
    "NA_D_70",
    "NA_B_22",
    "NA_B_21",
    "NA_S_13",
    "NA_R_6",
    "NA_S_12",
    "NA_D_54",
    "NA_S_6",
    "NA_B_11",
    "last-ave_R_23",
    "NA_B_1",
    "NA_D_39",
    "slope_D_110",
    "NA_S_2",
    "slope_D_109",
    "last-ave_D_86",
    "last-ave_D_87",
    "last-ave_R_18",
    "last-ave_D_88",
    "cat_first_D_116",
    "NA_R_1",
    "last-ave_D_93",
    "last-ave_D_94",
    "slope_D_94",
    "slope_D_93",
    "slope_R_23",
    "last-ave_D_109",
    "slope_D_88",
    "slope_R_18",
    "slope_D_87",
    "NA_B_2",
    "slope_B_39",
    "NA_S_5",
    "R_8_min",
    "NA_B_10",
    "NA_R_3",
    "NA_B_9",
    "NA_D_51",
    "R_10_min",
    "slope_B_42",
    "NA_B_7",
    "NA_B_6",
    "NA_D_49",
    "NA_D_47",
    "NA_D_41",
    "R_7_min",
    "NA_R_2",
    "NA_B_5",
    "NA_D_45",
    "NA_B_4",
    "NA_D_44",
    "D_116_last",
    "D_116_nunique",
    "NA_B_3",
    "NA_D_79",
    "NA_R_8",
    "NA_R_9",
    "NA_B_40",
    "NA_D_120",
    "first_R_18",
    "first_D_88",
    "NA_D_117",
    "NA_D_116",
    "first_B_31",
    "NA_D_114",
    "NA_D_113",
    "first_R_19",
    "NA_D_112",
    "NA_D_122",
    "NA_D_111",
    "NA_D_109",
    "NA_D_108",
    "NA_B_38",
    "NA_R_26",
    "NA_B_37",
    "NA_B_36",
    "NA_D_107",
    "NA_D_106",
    "first_R_17",
    "NA_D_123",
    "first_R_22",
    "R_14_min",
    "NA_D_143",
    "R_15_min",
    "first_R_8",
    "NA_D_140",
    "NA_D_139",
    "NA_D_138",
    "cat_-2_D_116",
    "NA_D_136",
    "NA_D_135",
    "NA_R_28",
    "NA_D_124",
    "first_R_13",
    "first_R_15",
    "NA_B_42",
    "NA_B_41",
    "NA_D_129",
    "first_D_87",
    "NA_D_127",
    "NA_D_126",
    "NA_D_125",
    "NA_D_103",
    "NA_S_26",
    "NA_S_16",
    "NA_D_83",
    "NA_D_87",
    "NA_D_86",
    "NA_S_18",
    "NA_B_30",
    "first_D_94",
    "NA_R_16",
    "NA_D_84",
    "NA_R_15",
    "NA_R_14",
    "NA_R_13",
    "NA_R_18",
    "NA_B_28",
    "NA_R_12",
    "NA_S_17",
    "NA_D_82",
    "NA_D_81",
    "NA_B_27",
    "NA_R_11",
    "NA_R_10",
    "NA_D_80",
    "NA_R_17",
    "first_D_93",
    "NA_S_25",
    "NA_D_91",
    "first_R_23",
    "NA_S_23",
    "NA_S_22",
    "NA_D_96",
    "NA_R_25",
    "NA_R_24",
    "NA_D_94",
    "NA_D_93",
    "NA_D_92",
    "NA_R_23",
    "NA_B_31",
    "NA_R_22",
    "NA_D_89",
    "NA_B_33",
    "NA_R_21",
    "NA_R_20",
    "NA_S_20",
    "NA_B_32",
    "NA_R_19",
    "NA_S_19",
    "NA_D_137",
]
useless_3 = [
    "D_136_min",
    "S_18_last",
    "R_23_min",
    "R_23_max",
    "R_23_last",
    "D_137_min",
    "D_92_min",
    "D_92_last",
    "D_87_std",
    "D_93_std",
    "D_87_mean",
    "B_31_max",
    "last/ave_D_92",
    "D_93_min",
    "D_87_min",
    "last/ave_D_93",
    "D_93_max",
    "D_93_last",
    "D_94_mean",
    "S_18_min",
    "D_65_min",
    "D_94_min",
    "D_94_max",
    "D_94_last",
    "first_R_4",
    "last/ave_D_94",
    "R_24_min",
    "last/ave_D_109",
    "R_23_std",
    "R_23_mean",
    "D_87_max",
    "R_20_min",
    "R_4_min",
    "last/ave_R_18",
    "D_88_min",
    "last/ave_R_28",
    "last/ave_D_87",
    "last/ave_S_18",
    "R_5_min",
    "D_88_mean",
    "R_18_last",
    "S_20_min",
    "R_18_max",
    "R_18_min",
    "R_18_std",
    "last/ave_D_88",
    "R_18_mean",
    "D_138_min",
    "R_21_min",
    "last/ave_R_22",
    "D_89_min",
    "S_6_min",
    "D_139_last",
    "R_17_min",
    "R_22_min",
    "D_87_last",
    "R_22_last",
    "last/ave_R_23",
    "R_24_last",
    "last/first_R_25",
    "last/first_R_28",
    "D_111_min",
    "D_123_min",
    "last-first_D_109",
    "last-first_D_110",
    "diff_last_-2_B_41",
    "diff_last_-2_D_127",
    "last-first_B_42",
    "last-first_R_28",
    "B_41_min",
    "B_42_std",
    "B_42_max",
    "D_109_last",
    "D_109_max",
    "D_109_min",
    "D_109_std",
    "D_109_mean",
    "diff_last_-2_D_111",
    "D_108_min",
    "diff_last_-2_D_109",
    "last-first_D_96",
    "diff_last_-2_R_28",
    "last-first_R_25",
    "last-first_R_24",
    "date_month_last",
    "D_127_min",
    "date_month_min",
    "D_125_min",
    "last-first_R_15",
    "last-first_D_87",
    "last-first_R_18",
    "last-first_D_88",
    "last-first_R_19",
    "last-first_S_20",
    "diff_last_-2_D_145",
    "diff_last_-2_D_143",
    "last-first_R_22",
    "last-first_R_23",
    "diff_last_-2_D_139",
    "last-first_D_93",
    "last-first_D_94",
    "diff_last_-2_D_108",
    "diff_last_-2_R_26",
    "R_25_min",
    "diff_last_-2_D_86",
    "last/first_R_23",
    "diff_last_-2_R_19",
    "last/first_D_93",
    "diff_last_-2_B_31",
    "diff_last_-2_D_88",
    "diff_last_-2_R_18",
    "diff_last_-2_R_17",
    "diff_last_-2_D_87",
    "last/first_D_94",
    "R_2_min",
    "D_135_min",
    "diff_last_-2_R_13",
    "last/first_D_109",
    "last/first_D_110",
    "diff_last_-2_D_82",
    "diff_last_-2_D_81",
    "D_96_min",
    "R_25_last",
    "diff_last_-2_R_21",
    "last/first_R_22",
    "diff_last_-2_D_89",
    "diff_last_-2_R_22",
    "R_28_mean",
    "R_28_std",
    "R_28_min",
    "R_28_max",
    "R_28_last",
    "last/first_R_15",
    "last/first_S_18",
    "last/first_D_87",
    "last/first_R_17",
    "last/first_R_18",
    "diff_last_-2_D_96",
    "diff_last_-2_R_25",
    "last/first_D_88",
    "diff_last_-2_D_94",
    "diff_last_-2_D_93",
    "diff_last_-2_D_92",
    "diff_last_-2_R_23",
    "R_16_min",
    "last-ave_R_28",
    "NA_D_145",
    "R_15_last",
    "NA_B_18",
    "R_13_min",
    "NA_B_16",
    "NA_D_65",
    "NA_D_64",
    "NA_D_63",
    "first_D_108",
    "NA_S_11",
    "NA_B_15",
    "first_D_109",
    "NA_D_60",
    "NA_D_59",
    "NA_B_14",
    "first_D_111",
    "NA_D_58",
    "NA_R_5",
    "slope_R_28",
    "NA_S_8",
    "NA_B_12",
    "first_R_28",
    "NA_R_4",
    "NA_B_19",
    "NA_D_66",
    "NA_B_20",
    "NA_P_4",
    "NA_D_78",
    "NA_B_26",
    "NA_B_25",
    "first_R_25",
    "NA_R_7",
    "NA_B_24",
    "first_D_96",
    "NA_D_75",
    "NA_D_74",
    "NA_B_23",
    "NA_D_68",
    "NA_S_15",
    "NA_D_72",
    "NA_D_71",
    "NA_D_70",
    "NA_B_22",
    "NA_B_21",
    "NA_S_13",
    "NA_R_6",
    "NA_S_12",
    "NA_D_54",
    "NA_S_6",
    "NA_B_11",
    "last-ave_R_23",
    "NA_B_1",
    "NA_D_39",
    "slope_D_110",
    "NA_S_2",
    "slope_D_109",
    "last-ave_D_86",
    "last-ave_D_87",
    "last-ave_R_18",
    "last-ave_D_88",
    "cat_first_D_116",
    "NA_R_1",
    "last-ave_D_93",
    "last-ave_D_94",
    "slope_D_94",
    "slope_D_93",
    "slope_R_23",
    "last-ave_D_109",
    "slope_D_88",
    "slope_R_18",
    "slope_D_87",
    "NA_B_2",
    "slope_B_39",
    "NA_S_5",
    "R_8_min",
    "NA_B_10",
    "NA_R_3",
    "NA_B_9",
    "NA_D_51",
    "R_10_min",
    "slope_B_42",
    "NA_B_7",
    "NA_B_6",
    "NA_D_49",
    "NA_D_47",
    "NA_D_41",
    "R_7_min",
    "NA_R_2",
    "NA_B_5",
    "NA_D_45",
    "NA_B_4",
    "NA_D_44",
    "D_116_last",
    "D_116_nunique",
    "NA_B_3",
    "NA_D_79",
    "NA_R_8",
    "NA_R_9",
    "NA_B_40",
    "NA_D_120",
    "first_R_18",
    "first_D_88",
    "NA_D_117",
    "NA_D_116",
    "first_B_31",
    "NA_D_114",
    "NA_D_113",
    "first_R_19",
    "NA_D_112",
    "NA_D_122",
    "NA_D_111",
    "NA_D_109",
    "NA_D_108",
    "NA_B_38",
    "NA_R_26",
    "NA_B_37",
    "NA_B_36",
    "NA_D_107",
    "NA_D_106",
    "first_R_17",
    "NA_D_123",
    "first_R_22",
    "R_14_min",
    "NA_D_143",
    "R_15_min",
    "first_R_8",
    "NA_D_140",
    "NA_D_139",
    "NA_D_138",
    "cat_-2_D_116",
    "NA_D_136",
    "NA_D_135",
    "NA_R_28",
    "NA_D_124",
    "first_R_13",
    "first_R_15",
    "NA_B_42",
    "NA_B_41",
    "NA_D_129",
    "first_D_87",
    "NA_D_127",
    "NA_D_126",
    "NA_D_125",
    "NA_D_103",
    "NA_S_26",
    "NA_S_16",
    "NA_D_83",
    "NA_D_87",
    "NA_D_86",
    "NA_S_18",
    "NA_B_30",
    "first_D_94",
    "NA_R_16",
    "NA_D_84",
    "NA_R_15",
    "NA_R_14",
    "NA_R_13",
    "NA_R_18",
    "NA_B_28",
    "NA_R_12",
    "NA_S_17",
    "NA_D_82",
    "NA_D_81",
    "NA_B_27",
    "NA_R_11",
    "NA_R_10",
    "NA_D_80",
    "NA_R_17",
    "first_D_93",
    "NA_S_25",
    "NA_D_91",
    "first_R_23",
    "NA_S_23",
    "NA_S_22",
    "NA_D_96",
    "NA_R_25",
    "NA_R_24",
    "NA_D_94",
    "NA_D_93",
    "NA_D_92",
    "NA_R_23",
    "NA_B_31",
    "NA_R_22",
    "NA_D_89",
    "NA_B_33",
    "NA_R_21",
    "NA_R_20",
    "NA_S_20",
    "NA_B_32",
    "NA_R_19",
    "NA_S_19",
    "NA_D_137",
]

useless_4 = [
    "D_136_min",
    "S_18_last",
    "R_23_min",
    "R_23_max",
    "R_23_last",
    "D_137_min",
    "D_92_min",
    "D_92_last",
    "D_87_std",
    "D_93_std",
    "D_87_mean",
    "B_31_max",
    "last/ave_D_92",
    "D_93_min",
    "D_87_min",
    "last/ave_D_93",
    "D_93_max",
    "D_93_last",
    "D_94_mean",
    "S_18_min",
    "D_65_min",
    "D_94_min",
    "D_94_max",
    "D_94_last",
    "first_R_4",
    "last/ave_D_94",
    "R_24_min",
    "last/ave_D_109",
    "R_23_std",
    "R_23_mean",
    "D_87_max",
    "R_20_min",
    "R_4_min",
    "last/ave_R_18",
    "D_88_min",
    "last/ave_R_28",
    "last/ave_D_87",
    "last/ave_S_18",
    "R_5_min",
    "D_88_mean",
    "R_18_last",
    "S_20_min",
    "R_18_max",
    "R_18_min",
    "R_18_std",
    "last/ave_D_88",
    "R_18_mean",
    "D_138_min",
    "R_21_min",
    "last/ave_R_22",
    "D_89_min",
    "S_6_min",
    "D_139_last",
    "R_17_min",
    "R_22_min",
    "D_87_last",
    "R_22_last",
    "last/ave_R_23",
    "R_24_last",
    "last/first_R_25",
    "last/first_R_28",
    "D_111_min",
    "D_123_min",
    "last-first_D_109",
    "last-first_D_110",
    "diff_last_-2_B_41",
    "diff_last_-2_D_127",
    "last-first_B_42",
    "last-first_R_28",
    "B_41_min",
    "B_42_std",
    "B_42_max",
    "D_109_last",
    "D_109_max",
    "D_109_min",
    "D_109_std",
    "D_109_mean",
    "diff_last_-2_D_111",
    "D_108_min",
    "diff_last_-2_D_109",
    "last-first_D_96",
    "diff_last_-2_R_28",
    "last-first_R_25",
    "last-first_R_24",
    "date_month_last",
    "D_127_min",
    "date_month_min",
    "D_125_min",
    "last-first_R_15",
    "last-first_D_87",
    "last-first_R_18",
    "last-first_D_88",
    "last-first_R_19",
    "last-first_S_20",
    "diff_last_-2_D_145",
    "diff_last_-2_D_143",
    "last-first_R_22",
    "last-first_R_23",
    "diff_last_-2_D_139",
    "last-first_D_93",
    "last-first_D_94",
    "diff_last_-2_D_108",
    "diff_last_-2_R_26",
    "R_25_min",
    "diff_last_-2_D_86",
    "last/first_R_23",
    "diff_last_-2_R_19",
    "last/first_D_93",
    "diff_last_-2_B_31",
    "diff_last_-2_D_88",
    "diff_last_-2_R_18",
    "diff_last_-2_R_17",
    "diff_last_-2_D_87",
    "last/first_D_94",
    "R_2_min",
    "D_135_min",
    "diff_last_-2_R_13",
    "last/first_D_109",
    "last/first_D_110",
    "diff_last_-2_D_82",
    "diff_last_-2_D_81",
    "D_96_min",
    "R_25_last",
    "diff_last_-2_R_21",
    "last/first_R_22",
    "diff_last_-2_D_89",
    "diff_last_-2_R_22",
    "R_28_mean",
    "R_28_std",
    "R_28_min",
    "R_28_max",
    "R_28_last",
    "last/first_R_15",
    "last/first_S_18",
    "last/first_D_87",
    "last/first_R_17",
    "last/first_R_18",
    "diff_last_-2_D_96",
    "diff_last_-2_R_25",
    "last/first_D_88",
    "diff_last_-2_D_94",
    "diff_last_-2_D_93",
    "diff_last_-2_D_92",
    "diff_last_-2_R_23",
    "R_16_min",
    "last-ave_R_28",
    "NA_D_145",
    "R_15_last",
    "NA_B_18",
    "R_13_min",
    "NA_B_16",
    "NA_D_65",
    "NA_D_64",
    "NA_D_63",
    "first_D_108",
    "NA_S_11",
    "NA_B_15",
    "first_D_109",
    "NA_D_60",
    "NA_D_59",
    "NA_B_14",
    "first_D_111",
    "NA_D_58",
    "NA_R_5",
    "slope_R_28",
    "NA_S_8",
    "NA_B_12",
    "first_R_28",
    "NA_R_4",
    "NA_B_19",
    "NA_D_66",
    "NA_B_20",
    "NA_P_4",
    "NA_D_78",
    "NA_B_26",
    "NA_B_25",
    "first_R_25",
    "NA_R_7",
    "NA_B_24",
    "first_D_96",
    "NA_D_75",
    "NA_D_74",
    "NA_B_23",
    "NA_D_68",
    "NA_S_15",
    "NA_D_72",
    "NA_D_71",
    "NA_D_70",
    "NA_B_22",
    "NA_B_21",
    "NA_S_13",
    "NA_R_6",
    "NA_S_12",
    "NA_D_54",
    "NA_S_6",
    "NA_B_11",
    "last-ave_R_23",
    "NA_B_1",
    "NA_D_39",
    "slope_D_110",
    "NA_S_2",
    "slope_D_109",
    "last-ave_D_86",
    "last-ave_D_87",
    "last-ave_R_18",
    "last-ave_D_88",
    "cat_first_D_116",
    "NA_R_1",
    "last-ave_D_93",
    "last-ave_D_94",
    "slope_D_94",
    "slope_D_93",
    "slope_R_23",
    "last-ave_D_109",
    "slope_D_88",
    "slope_R_18",
    "slope_D_87",
    "NA_B_2",
    "slope_B_39",
    "NA_S_5",
    "R_8_min",
    "NA_B_10",
    "NA_R_3",
    "NA_B_9",
    "NA_D_51",
    "R_10_min",
    "slope_B_42",
    "NA_B_7",
    "NA_B_6",
    "NA_D_49",
    "NA_D_47",
    "NA_D_41",
    "R_7_min",
    "NA_R_2",
    "NA_B_5",
    "NA_D_45",
    "NA_B_4",
    "NA_D_44",
    "D_116_last",
    "D_116_nunique",
    "NA_B_3",
    "NA_D_79",
    "NA_R_8",
    "NA_R_9",
    "NA_B_40",
    "NA_D_120",
    "first_R_18",
    "first_D_88",
    "NA_D_117",
    "NA_D_116",
    "first_B_31",
    "NA_D_114",
    "NA_D_113",
    "first_R_19",
    "NA_D_112",
    "NA_D_122",
    "NA_D_111",
    "NA_D_109",
    "NA_D_108",
    "NA_B_38",
    "NA_R_26",
    "NA_B_37",
    "NA_B_36",
    "NA_D_107",
    "NA_D_106",
    "first_R_17",
    "NA_D_123",
    "first_R_22",
    "R_14_min",
    "NA_D_143",
    "R_15_min",
    "first_R_8",
    "NA_D_140",
    "NA_D_139",
    "NA_D_138",
    "cat_-2_D_116",
    "NA_D_136",
    "NA_D_135",
    "NA_R_28",
    "NA_D_124",
    "first_R_13",
    "first_R_15",
    "NA_B_42",
    "NA_B_41",
    "NA_D_129",
    "first_D_87",
    "NA_D_127",
    "NA_D_126",
    "NA_D_125",
    "NA_D_103",
    "NA_S_26",
    "NA_S_16",
    "NA_D_83",
    "NA_D_87",
    "NA_D_86",
    "NA_S_18",
    "NA_B_30",
    "first_D_94",
    "NA_R_16",
    "NA_D_84",
    "NA_R_15",
    "NA_R_14",
    "NA_R_13",
    "NA_R_18",
    "NA_B_28",
    "NA_R_12",
    "NA_S_17",
    "NA_D_82",
    "NA_D_81",
    "NA_B_27",
    "NA_R_11",
    "NA_R_10",
    "NA_D_80",
    "NA_R_17",
    "first_D_93",
    "NA_S_25",
    "NA_D_91",
    "first_R_23",
    "NA_S_23",
    "NA_S_22",
    "NA_D_96",
    "NA_R_25",
    "NA_R_24",
    "NA_D_94",
    "NA_D_93",
    "NA_D_92",
    "NA_R_23",
    "NA_B_31",
    "NA_R_22",
    "NA_D_89",
    "NA_B_33",
    "NA_R_21",
    "NA_R_20",
    "NA_S_20",
    "NA_B_32",
    "NA_R_19",
    "NA_S_19",
    "NA_D_137",
]
all_useless = useless_0 + useless_1 + useless_2 + useless_3 + useless_4
print(len(all_useless))

In [ ]:
all_useless = list(set(all_useless))

In [ ]:
train = pd.read_parquet("Data/test_all_slopes_corr_pcaslope_lagv2_avediff_catLastLastNAdate_part1.parquet")
print(train.shape)
train = train.drop(all_useless, axis=1)
print(train.shape)

In [ ]:
corr_col = train.columns[train.columns.str.startswith("corr_")].to_list()

top_corr = [
    "corr_D_39-B_26",
    "corr_D_48-B_4",
    "corr_P_2-D_44",
    "corr_D_47-B_4",
    "corr_D_47-D_39",
    "corr_P_2-B_4",
    "corr_D_39-B_10",
    "corr_D_44-B_4",
    "corr_D_39-B_2",
    "corr_D_46-B_4",
    "corr_D_48-B_3",
    "corr_D_48-B_9",
    "corr_S_5-S_24",
    "corr_S_7-S_3",
    "corr_D_43-D_144",
    "corr_D_48-D_39",
    "corr_P_3-D_46",
    "corr_S_5-D_43",
    "corr_R_1-B_4",
    "corr_P_3-D_47",
    "corr_D_39-B_3",
    "corr_R_6-D_39",
    "corr_S_27-B_2",
    "corr_S_23-D_43",
    "corr_R_6-D_69",
    "corr_P_2-D_48",
    "corr_S_25-B_4",
    "corr_D_43-B_4",
    "corr_R_27-D_69",
    "corr_S_7-S_27",
    "corr_D_39-B_11",
    "corr_S_3-D_39",
    "corr_S_12-B_4",
    "corr_D_39-B_15",
    "corr_R_27-B_26",
    "corr_S_23-D_39",
    "corr_R_27-R_1",
    "corr_R_1-D_39",
    "corr_S_19-D_39",
    "corr_S_27-B_3",
    "corr_S_16-D_39",
    "corr_R_27-B_5",
    "corr_S_3-D_62",
    "corr_D_71-D_62",
    "corr_R_27-D_39",
    "corr_D_48-D_43",
    "corr_D_61-B_36",
    "corr_S_25-D_39",
    "corr_R_6-D_43",
    "corr_S_27-R_27",
    "corr_S_27-S_12",
    "corr_S_27-D_39",
    "corr_D_46-B_3",
    "corr_D_62-D_47",
    "corr_B_4-B_3",
    "corr_R_1-D_48",
]

corr_to_remove = set(corr_col).difference(set(top_corr))
train.drop(corr_to_remove, axis=1, inplace=True)
train.shape

In [ ]:
all.head()

In [ ]:
train.head()

In [ ]:
dataset_train = pd.concat([train, all.loc[train.index]], axis=1)
dataset_train.shape

In [ ]:
dataset_train.to_parquet("Data/test_all_features_part1.parquet")

In [ ]:
train = pd.read_parquet("Data/test_all_slopes_corr_pcaslope_lagv2_avediff_catLastLastNAdate_part2.parquet")
print(train.shape)
train = train.drop(all_useless, axis=1)
print(train.shape)

In [ ]:
corr_col = train.columns[train.columns.str.startswith("corr_")].to_list()

top_corr = [
    "corr_D_39-B_26",
    "corr_D_48-B_4",
    "corr_P_2-D_44",
    "corr_D_47-B_4",
    "corr_D_47-D_39",
    "corr_P_2-B_4",
    "corr_D_39-B_10",
    "corr_D_44-B_4",
    "corr_D_39-B_2",
    "corr_D_46-B_4",
    "corr_D_48-B_3",
    "corr_D_48-B_9",
    "corr_S_5-S_24",
    "corr_S_7-S_3",
    "corr_D_43-D_144",
    "corr_D_48-D_39",
    "corr_P_3-D_46",
    "corr_S_5-D_43",
    "corr_R_1-B_4",
    "corr_P_3-D_47",
    "corr_D_39-B_3",
    "corr_R_6-D_39",
    "corr_S_27-B_2",
    "corr_S_23-D_43",
    "corr_R_6-D_69",
    "corr_P_2-D_48",
    "corr_S_25-B_4",
    "corr_D_43-B_4",
    "corr_R_27-D_69",
    "corr_S_7-S_27",
    "corr_D_39-B_11",
    "corr_S_3-D_39",
    "corr_S_12-B_4",
    "corr_D_39-B_15",
    "corr_R_27-B_26",
    "corr_S_23-D_39",
    "corr_R_27-R_1",
    "corr_R_1-D_39",
    "corr_S_19-D_39",
    "corr_S_27-B_3",
    "corr_S_16-D_39",
    "corr_R_27-B_5",
    "corr_S_3-D_62",
    "corr_D_71-D_62",
    "corr_R_27-D_39",
    "corr_D_48-D_43",
    "corr_D_61-B_36",
    "corr_S_25-D_39",
    "corr_R_6-D_43",
    "corr_S_27-R_27",
    "corr_S_27-S_12",
    "corr_S_27-D_39",
    "corr_D_46-B_3",
    "corr_D_62-D_47",
    "corr_B_4-B_3",
    "corr_R_1-D_48",
]

corr_to_remove = set(corr_col).difference(set(top_corr))
train.drop(corr_to_remove, axis=1, inplace=True)
train.shape

In [ ]:
dataset_train = pd.concat([train, all.loc[train.index]], axis=1)
dataset_train.shape

In [ ]:
dataset_train.to_parquet("Data/test_all_features_part2.parquet")